# Neural Collaborative Filtering  

## 1. Dataset: Movie lens  

- data description: https://files.grouplens.org/datasets/movielens/ml-1m-README.txt

In [29]:
import pandas as pd
import scipy.sparse as sp

data_dir = 'dataset/'

In [37]:
train_df = pd.read_csv(data_dir + "ml-1m.train.rating", sep='\t', header=None)
train_df.columns = ["userID", "itemID", "rating","timestamp"]
print(len(train_df))
train_df.head()

994169


,userID,itemID,rating,timestamp
0,0,32,4,978824330
1,0,34,4,978824330
2,0,4,5,978824291
3,0,35,4,978824291
4,0,30,4,978824291


- 해당 연구에서는 user-item interaction만 사용 (rating 불필요)  

In [26]:
test_df = pd.read_csv(data_dir + "ml-1m.test.negative", sep='\t', header=None)
print(len(test_df))
test_df.head()

6040


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,"(0,25)",1064,174,2791,3373,269,2678,1902,3641,1216,...,2854,3067,58,2551,2333,2688,3703,1300,1924,3118
1,"(1,133)",1072,3154,3368,3644,549,1810,937,1514,1713,...,1535,341,3525,1429,2225,1628,2061,469,3056,2553
2,"(2,207)",2216,209,2347,3,1652,3397,383,2905,2284,...,953,865,813,1353,2945,2580,2989,2790,2879,2481
3,"(3,208)",3023,1489,1916,1706,1221,1191,2671,81,2483,...,3347,1707,2901,2767,2167,1921,247,1618,2016,2323
4,"(4,222)",1794,3535,108,593,466,2048,854,1378,1301,...,2490,1332,2526,2804,2027,833,176,463,2851,2453


- negative sampling data

In [47]:
def load_all():
    train_df = pd.read_csv(data_dir + "ml-1m.train.rating", sep='\t', header=None)
    train_df.columns = ["userID", "itemID", "rating","timestamp"]
    train_df = train_df[["userID","itemID"]] 

    user_num = train_df["userID"].max() + 1
    item_num = train_df["itemID"].max() + 1

    # dok matrix 생성 (Dictionary Of Key)
    train_df_list = train_df.values.tolist()
    train_mat = sp.dok_matrix((user_num, item_num), dtype=np.float32)
    for x in train_df_list:
        train_mat[x[0], x[1]] = 1.0

    test_df = pd.read_csv(data_dir + "ml-1m.test.negative", sep='\t', header=None)
    test_df = test_df.values.tolist()
    test_df_list=[]
    for arr in test_df:
        _user,_item = eval(arr[0])
        test_df_list.append([_user,_item])
        for j in arr[1:]:
            test_df_list.append([_user,int(j)])

    return train_df_list, test_df_list, user_num, item_num, train_mat

# prepare dataset
train_data, test_data, user_num, item_num, train_mat = load_all()

In [53]:
print("user:", user_num)
print("iser:", item_num)

user: 6040
iser: 3706


In [103]:
print(train_mat)

  (0, 32)	1.0
  (0, 34)	1.0
  (0, 4)	1.0
  (0, 35)	1.0
  (0, 30)	1.0
  (0, 29)	1.0
  (0, 33)	1.0
  (0, 40)	1.0
  (0, 10)	1.0
  (0, 16)	1.0
  (0, 23)	1.0
  (0, 28)	1.0
  (0, 12)	1.0
  (0, 8)	1.0
  (0, 5)	1.0
  (0, 20)	1.0
  (0, 46)	1.0
  (0, 15)	1.0
  (0, 50)	1.0
  (0, 49)	1.0
  (0, 13)	1.0
  (0, 1)	1.0
  (0, 52)	1.0
  (0, 38)	1.0
  (0, 19)	1.0
  :	:
  (6039, 309)	1.0
  (6039, 1951)	1.0
  (6039, 51)	1.0
  (6039, 104)	1.0
  (6039, 134)	1.0
  (6039, 2585)	1.0
  (6039, 3042)	1.0
  (6039, 2471)	1.0
  (6039, 167)	1.0
  (6039, 559)	1.0
  (6039, 23)	1.0
  (6039, 420)	1.0
  (6039, 3412)	1.0
  (6039, 9)	1.0
  (6039, 346)	1.0
  (6039, 1234)	1.0
  (6039, 1615)	1.0
  (6039, 2748)	1.0
  (6039, 1551)	1.0
  (6039, 2517)	1.0
  (6039, 1092)	1.0
  (6039, 41)	1.0
  (6039, 128)	1.0
  (6039, 323)	1.0
  (6039, 669)	1.0


## 2. DataLoader  